## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[requests_st_support]]

[[intercoms_dir_asgard]]

[[intercoms_st_partner]]

[[companies_dir_partner]]

[[categories_st_support]]
___

In [2]:
query_text = """--sql
CREATE TABLE db1.t_requests_info_st_support
    (
        `report_date` Date,
        `ticket_id` Int32,
        `intercom_id` String,
        `intercom_uuid` String,
        `camera_id` String,
        `server_stream_url` String,
        `detailed_category` String,
        `category` String,
        `subcategory` String,
        `maincategory` String,  
        `version_os` String,
        `version_app` String,
        `partner_uuid` String,
        `company_name` String,
        `partner_lk` String,
        `com_d_par.company_name` String,
        `device_type` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [4]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_requests_info_st_support_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_requests_info_st_support AS
WITH t1 AS(
SELECT 
    req_s_sup.`report_date` AS report_date, 
    `ticket_id`, 
    `intercom_id`, 
    int_d_as.`intercom_uuid` AS intercom_uuid,
    `camera_id`,
    `server_stream_url`, 
    replaceAll(trim(arrayJoin(splitByChar(',',`category`))),'.','') AS `category`, 
    `version_os`, 
    `version_app`,
    int_s_par.`partner_uuid` AS partner_uuid,
    `company_name`,
    `partner_lk`,
    `device_type`
FROM `db1`.`requests_st_support_ch` AS req_s_sup
LEFT JOIN db1.intercoms_dir_asgard_ch AS int_d_as ON int_d_as.motherboard_id = req_s_sup.intercom_id
LEFT JOIN db1.intercoms_st_partner_ch AS int_s_par 
    ON  int_s_par.intercom_uuid = int_d_as.`intercom_uuid`
    AND int_s_par.report_date = req_s_sup.`report_date`
LEFT JOIN db1.companies_dir_partner_ch AS com_d_par
    ON int_s_par.partner_uuid = com_d_par.partner_uuid
   	),
--
t2 AS (
SELECT
	report_date,
	ticket_id,
	intercom_id,
	intercom_uuid,
	camera_id,
    server_stream_url,
    category,
    if(length(splitByChar(':',category)) = 2, trim(splitByChar(':',category)[2]),trim(splitByChar(':',category)[1])) AS `detailed_category`,
    version_os, 
    version_app,
    partner_uuid,
    company_name,
    partner_lk,
    device_type
FROM t1
)
--
SELECT
	t2.report_date AS report_date,
	ticket_id,
	intercom_id,
	intercom_uuid,
	camera_id,
    server_stream_url,
    category,
    `detailed_category`,
    if(splitByChar('|',Category)[2] = '','Main',splitByChar('|',Category)[2])  AS `subcategory`, 
    splitByChar('|',Category)[1] AS `maincategory`, 
    version_os, 
    version_app,
    partner_uuid,
    company_name,
    partner_lk,
    device_type
FROM t2
JOIN db1.categories_st_support_ch AS cat_s_sup 
	ON cat_s_sup.report_date = t2.report_date
	AND lowerUTF8(cat_s_sup.Details) = lowerUTF8(t2.detailed_category)
WHERE  lowerUTF8(subcategory) = 'info'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [7]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_requests_info_st_support
    ORDER BY report_date
    limit 10
    """

ch.query_run(query_text)


report_date,ticket_id,intercom_id,intercom_uuid,camera_id,server_stream_url,detailed_category,category,subcategory,maincategory,version_os,version_app,partner_uuid,company_name,partner_lk,com_d_par.company_name,device_type
date,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2025-07-20,196533,"""1944104891""","""5d47ccf8-e8dc-499a-b4b0-acae31…","""""","""msk-v04.sputnik.systems""","""удалить из семьи/лишние пользо…","""MOB|Info: удалить из семьи/лиш…","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""bf6984b9-92bc-4752-8f14-274ae6…","""ООО ""Комфорт Сервис"" ИНН 23101…","""120506""","""""","""intercom"""
2025-07-20,196491,"""2121124608""","""390971b1-be51-43ef-8f57-95284b…","""""","""msk-v06.sputnik.systems""","""добавить виджет""","""MOB|Info: добавить виджет""","""Info""","""MOB""","""Android""","""6.21.1(417)""","""5ba3ca92-4ee9-4d27-9ac2-e0b569…","""ООО «Технический центр домофон…","""123730""","""""","""intercom"""
2025-07-20,196470,"""2236151454""","""a0496ee5-b323-44fa-85eb-22ad78…","""""","""msk-v01.sputnik.systems""","""проблема с оплатой""","""MOB|Info: проблема с оплатой""","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""1c64b848-e82c-46ee-9d0d-c74093…","""ООО Добрыня ПРО""","""121833""","""""","""intercom"""
2025-07-20,196471,"""2235148552""","""7499ad33-2f94-4d06-9c03-ac539b…","""""","""msk-v01.sputnik.systems""","""кто администратор""","""MOB|Info: кто администратор""","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""8e9d0d2c-8798-4eba-8e8d-6fa5e0…","""ABК-ВЕЛЛКОМСТРОЙ ООО""","""120189""","""""","""intercom"""
2025-07-20,196503,"""2108118655""","""ae91c1b4-56ed-42a3-a48b-b1def3…","""""","""v08.sputnik.systems""","""функции в подписке""","""MOB|Info: функции в подписке""","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""0e7236ac-b8bd-4ab0-8634-d165ad…","""Цифрал Сервис УК""","""120345""","""""","""intercom"""
2025-07-20,196407,"""2121122914""","""198f79e2-86b7-4aee-ab25-bf326f…","""""","""msk-v06.sputnik.systems""","""оплатить домофон/лицевой счёт …","""MOB|Info: оплатить домофон/лиц…","""Info""","""MOB""","""Android""","""6.21.1(417)""","""0e7236ac-b8bd-4ab0-8634-d165ad…","""Цифрал Сервис УК""","""120345""","""""","""intercom"""
2025-07-20,196581,"""2236151922""","""ec2030da-cf11-4ca1-8a13-001511…","""""","""msk-v04.sputnik.systems""","""подключение архива""","""MOB|Info: подключение архива""","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""e08b9727-86e2-47e8-aaad-dff3c0…","""ИП Владимирова М.Н. ПРО""","""132773""","""""","""intercom"""
2025-07-20,196477,"""2235144423""","""cdf83ccb-41d1-4e5d-a55f-d85b71…","""""","""msk-v10.sputnik.systems""","""удалить из семьи/лишние пользо…","""MOB|Info: удалить из семьи/лиш…","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""ae55bd58-5c73-4ba6-8a8e-c46683…","""ИП Маргорин В.В.""","""123409""","""""","""intercom"""
2025-07-20,196477,"""2235144423""","""cdf83ccb-41d1-4e5d-a55f-d85b71…","""""","""msk-v10.sputnik.systems""","""сменить администратора""","""MOB|Info: сменить администрато…","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""ae55bd58-5c73-4ba6-8a8e-c46683…","""ИП Маргорин В.В.""","""123409""","""""","""intercom"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_requests_info_st_support DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.t_requests_info_st_support_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘


### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.t_requests_info_st_support
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_requests_info_st_support_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
